# Analysis of Spotify Tracks with Neural Networks
**A project by Graham Waters, prepared 2022-11-14**

Executive Summary

# Executive Summary



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



# Part 1: Using Clustering on Spotify data to predict genres of songs



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



# What Data is available in our datasets?



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



## Data Dictionary



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### File 1. data_by_genres.csv



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### File 2. spotify_artists.csv



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### File 3. spotify_tracks.csv



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### File 4. low_level_audio_features.csv



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### File 5. track_features.csv



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



## Comments about the Next Steps



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Get rid of columns that don't help predict genre



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Look into the pandas join function



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



## EDA - Exploratory Data Analysis



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Loading The Data from the Files



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



# import the data from the csv files into pandas dataframes (data/SpotGenTrack)



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



# FeaturesExtracted from the audio files



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### What are the most popular genres?



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### What are the most popular artists?



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### What correlations are there between the features?



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### What does the distribution of the features look like?



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### How do each of the features relate to the target (genre)?



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



## Data Preprocessing



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Missing data



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### types of data (categorical, numerical, ordinal)



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### standardize the columns (scale the data).



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



## Start building the models



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Gradient Boosting Model



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Random Forest Model



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Support Vector Machine potentially.



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



### Use A neural network (deep learning)



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



## Evaluate the models



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



# Part 2. Genre Generation with DBSSCAN Clustering



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```



# Components to this section of the project:



```
  
  
  
  | Part One | Part Two |
  |----------|----------|
  | [Part One](#part-one) | [Part Two](#part-two) |
  | Classification with NNets | Clustering with DBScan|
  
  # Table of Contents
  - [Analysis of Spotify Tracks with Neural Networks](#analysis-of-spotify-tracks-with-neural-networks)
  - [Table of Contents](#table-of-contents)
  - [Executive Summary](#executive-summary)
  - [Part 1: Using Clustering on Spotify data to predict genres of songs](#part-1-using-clustering-on-spotify-data-to-predict-genres-of-songs)
  - [What Data is available in our datasets?](#what-data-is-available-in-our-datasets)
    - [Data Dictionary](#data-dictionary)
  - [File 1. data_by_genres.csv](#file-1-data_by_genrescsv)
  - [File 2. spotify_artists.csv](#file-2-spotify_artistscsv)
  - [File 3. spotify_tracks.csv](#file-3-spotify_trackscsv)
  - [File 4. low_level_audio_features.csv](#file-4-low_level_audio_featurescsv)
  - [File 5. track_features.csv](#file-5-track_featurescsv)
    - [Comments about the Next Steps](#comments-about-the-next-steps)
  - [Get rid of columns that don't help predict genre](#get-rid-of-columns-that-dont-help-predict-genre)
  - [Look into the pandas join function](#look-into-the-pandas-join-function)
    - [EDA - Exploratory Data Analysis](#eda---exploratory-data-analysis)
  - [Loading The Data from the Files](#loading-the-data-from-the-files)
  - [What are the most popular genres?](#what-are-the-most-popular-genres)
  - [What are the most popular artists?](#what-are-the-most-popular-artists)
  - [What correlations are there between the features?](#what-correlations-are-there-between-the-features)
  - [What does the distribution of the features look like?](#what-does-the-distribution-of-the-features-look-like)
  - [How do each of the features relate to the target (genre)?](#how-do-each-of-the-features-relate-to-the-target-genre)
    - [Data Preprocessing](#data-preprocessing)
  - [Missing data](#missing-data)
  - [types of data (categorical, numerical, ordinal)](#types-of-data-categorical-numerical-ordinal)
  - [standardize the columns (scale the data).](#standardize-the-columns-scale-the-data)
  - [Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)](#encode-the-categorical-data-one-hot-encoding-use-get-dummies-vs-one-hot-encoding)
    - [Start building the models](#start-building-the-models)
  - [Gradient Boosting Model](#gradient-boosting-model)
  - [Random Forest Model](#random-forest-model)
  - [Support Vector Machine potentially.](#support-vector-machine-potentially)
  - [Use A neural network (deep learning)](#use-a-neural-network-deep-learning)
    - [Evaluate the models](#evaluate-the-models)
  - [Part 2. Genre Generation with DBSSCAN Clustering](#part-2-genre-generation-with-dbsscan-clustering)
  - [Components to this section of the project:](#components-to-this-section-of-the-project)
  
  # Executive Summary
  
  This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm.
  
  # Part 1: Using Clustering on Spotify data to predict genres of songs
  We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre.
  
  # What Data is available in our datasets?
  
  ## Data Dictionary
  
  ### File 1. data_by_genres.csv
  
  | Column Name | Description |
  |-------------|-------------|
  | genre | The genre of the song |
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 2. spotify_artists.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
  | followers | The number of followers the artist has. |
  | genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
  | id | The Spotify ID for the artist. |
  | name | The name of the artist. |
  | track_id | The Spotify ID for the track. |
  | track_name_prev | The name of the track. |
  | type | The object type: “artist” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/*
  
  ### File 3. spotify_tracks.csv
  
  
  | Column Name | Description |
  |-------------|-------------|
  | acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
  | album_id | The Spotify ID for the album. |
  | analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
  | artists_id | The Spotify ID for the artist. |
  | available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
  | country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
  | danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
  | disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
  | duration_ms | The duration of the track in milliseconds. |
  | energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
  | href | A link to the Web API endpoint providing full details of the track. |
  | id | The Spotify ID for the track. |
  | instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
  | key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
  | liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
  | loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
  | lyrics | The lyrics of the track. |
  | mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
  | name | The name of the track. |
  | playlist | The name of the playlist. |
  | popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
  | preview_url | A link to a 30 second preview (MP3 format) of the track. |
  | speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
  | tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
  | time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
  | track_href | A link to the Web API endpoint providing full details of the track. |
  | track_name_prev | The name of the track. |
  | track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
  | uri | The Spotify URI for the track. |
  | valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
  | type | The object type: “track” |
  
  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/*
  
  ### File 4. low_level_audio_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | Chroma_1 | Chroma feature 1 |
  | Chroma_10 | Chroma feature 10 |
  | Chroma_11 | Chroma feature 11 |
  | Chroma_12 | Chroma feature 12 |
  | Chroma_2 | Chroma feature 2 |
  | Chroma_3 | Chroma feature 3 |
  | Chroma_4 | Chroma feature 4 |
  | Chroma_5 | Chroma feature 5 |
  | Chroma_6 | Chroma feature 6 |
  | Chroma_7 | Chroma feature 7 |
  | Chroma_8 | Chroma feature 8 |
  | Chroma_9 | Chroma feature 9 |
  | MFCC_1 | MFCC feature 1 |
  | MFCC_10 | MFCC feature 10 |
  | MFCC_11 | MFCC feature 11 |
  | MFCC_12 | MFCC feature 12 |
  | MFCC_13 | MFCC feature 13 |
  | MFCC_14 | MFCC feature 14 |
  | MFCC_15 | MFCC feature 15 |
  | MFCC_16 | MFCC feature 16 |
  | MFCC_17 | MFCC feature 17 |
  | MFCC_18 | MFCC feature 18 |
  | MFCC_19 | MFCC feature 19 |
  | MFCC_2 | MFCC feature 2 |
  | MFCC_20 | MFCC feature 20 |
  | MFCC_3 | MFCC feature 3 |
  | MFCC_4 | MFCC feature 4 |
  | MFCC_5 | MFCC feature 5 |
  | MFCC_6 | MFCC feature 6 |
  | MFCC_7 | MFCC feature 7 |
  | MFCC_8 | MFCC feature 8 |
  | MFCC_9 | MFCC feature 9 |
  | Tonnetz_1 | Tonnetz feature 1 |
  | Tonnetz_2 | Tonnetz feature 2 |
  | Tonnetz_3 | Tonnetz feature 3 |
  | Tonnetz_4 | Tonnetz feature 4 |
  | Tonnetz_5 | Tonnetz feature 5 |
  | Tonnetz_6 | Tonnetz feature 6 |
  | ZCR | Zero Crossing Rate |
  | entropy_energy | Entropy of the energy |
  | spectral_bandwith | Spectral Bandwith |
  | spectral_centroid | Spectral Centroid |
  | spectral_rollOff_max | Spectral RollOff Max |
  | spectral_rollOff_min | Spectral RollOff Min |
  | track_id | Spotify track ID |
  
  *source:*
  
  ### File 5. track_features.csv
  
  
  
  | Column Name | Description |
  |-------------|-------------|
  | mean_syllables_word | Mean number of syllables per word |
  | mean_words_sentence | Mean number of words per sentence |
  | n_sentences | Number of sentences |
  | n_words | Number of words |
  | sentence_similarity | Sentence similarity |
  | track_id | Spotify track ID |
  | vocabulary_wealth | Vocabulary wealth |
  
  *source:*
  
  ## Comments about the Next Steps
  
  ### Get rid of columns that don't help predict genre
  ### Look into the pandas join function
  
  The end goal is to have single dataframe joining five tables on the track id column
  note: check if track id exists in all tables
  
  ## EDA - Exploratory Data Analysis
  Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.
  
  
  ### Loading The Data from the Files
  Using pandas to load the data from the files into dataframes.
  
  ```python
  # import the data from the csv files into pandas dataframes (data/SpotGenTrack)
  df_albums = pd.read_csv('data/SpotGenTrack/DataSources/spotify_albums.csv')
  df_artists = pd.read_csv('data/SpotGenTrack/DataSources/spotify_artists.csv')
  df_tracks = pd.read_csv('data/SpotGenTrack/DataSources/spotify_tracks.csv')
  # FeaturesExtracted from the audio files
  df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv')
  df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv')
  ```
```

